# HandsOn Week 13
Welcome to handsOn week 13: graph in distributed system, using GraphFrames. To use GraphFrames, you need to run ```pyspark --packages graphframes:graphframes:0.8.0-spark2.4-s_2.11``` (adjust with your Spark version. That script uses Spark 2.4.x, the one used in our VM, and it will download the graphframes package automatically, thus, make sure you have internet connection when launching the script).

Note: you can use any Spark & GraphFrames API (without building from-the-scratch).

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

23/05/08 11:37:30 WARN Utils: Your hostname, farras-HP-Laptop-14s-dk0xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.57 instead (on interface wlp3s0)
23/05/08 11:37:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/05/08 11:37:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/08 11:37:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/08 11:37:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Read The Dataset and Build The Graph

In [5]:
from pyspark.sql.types import *
from graphframes import *

def create_transport_graph(nodes_filePath, edges_filePath):
    node_fields = [
        StructField("id", StringType(), True),
        StructField("latitude", FloatType(), True),
        StructField("longitude", FloatType(), True),
        StructField("population", IntegerType(), True)
    ]
    nodes = spark.read.csv(nodes_filePath, header=True,
                           schema=StructType(node_fields))
    edge_fields = [
        StructField("src", StringType(), True),
        StructField("dst", StringType(), True),
        StructField("relationship", StringType(), True),
        StructField("cost", FloatType(), True)
    ]
    rels = spark.read.csv(edges_filePath, header=True,
                         schema=StructType(edge_fields))
    reversed_rels = (rels.withColumn("newSrc", rels.dst)
                     .withColumn("newDst", rels.src)
                     .drop("dst", "src")
                     .withColumnRenamed("newSrc", "src")
                     .withColumnRenamed("newDst", "dst")
                     .select("src", "dst", "relationship", "cost"))

    relationships = rels.union(reversed_rels)
    
    return GraphFrame(nodes, relationships)

g = create_transport_graph("./transport-nodes.csv", "./transport-relationships.csv")
g.vertices.show()
g.edges.show()

+----------------+---------+---------+----------+
|              id| latitude|longitude|population|
+----------------+---------+---------+----------+
|       Amsterdam| 52.37919| 4.899431|    821752|
|         Utrecht|52.092876|  5.10448|    334176|
|        Den Haag|52.078663| 4.288788|    514861|
|       Immingham| 53.61239| -0.22219|      9642|
|       Doncaster| 53.52285| -1.13116|    302400|
|Hoek van Holland|  51.9775|  4.13333|      9382|
|      Felixstowe| 51.96375|   1.3511|     23689|
|         Ipswich| 52.05917|  1.15545|    133384|
|      Colchester| 51.88921|  0.90421|    104390|
|          London|51.509865|-0.118092|   8787892|
|       Rotterdam|  51.9225|  4.47917|    623652|
|           Gouda| 52.01667|  4.70833|     70939|
+----------------+---------+---------+----------+

+----------------+----------------+------------+-----+
|             src|             dst|relationship| cost|
+----------------+----------------+------------+-----+
|       Amsterdam|         Utrecht

## Milestone 01
1. Find source-destination airport pair/pairs with maximum cost
2. Find source-destination airport pair/pairs with minimum cost
3. Calculate the average cost

In [13]:
# write your code here

#1. src-dst airport with max cost
maxCost = g.edges.groupBy().max("cost")
maxCost.show()
mx = 369.0
routeWithMaxCost = g.edges.filter(f"cost = {mx}")
routeWithMaxCost.show()

#2. src-dst airport with min cost
minCost = g.edges.groupBy().min("cost")
minCost.show()
mn = 22.0
routeWithMinCost = g.edges.filter(f"cost = {mn}")
routeWithMinCost.show()


#3. average cost
averageCost = g.edges.groupBy().avg("cost")
averageCost.show()


+---------+
|max(cost)|
+---------+
|    369.0|
+---------+

+---------+---------+------------+-----+
|      src|      dst|relationship| cost|
+---------+---------+------------+-----+
|Amsterdam|Immingham|       EROAD|369.0|
|Immingham|Amsterdam|       EROAD|369.0|
+---------+---------+------------+-----+

+---------+
|min(cost)|
+---------+
|     22.0|
+---------+

+----------+----------+------------+----+
|       src|       dst|relationship|cost|
+----------+----------+------------+----+
|   Ipswich|Felixstowe|       EROAD|22.0|
|Felixstowe|   Ipswich|       EROAD|22.0|
+----------+----------+------------+----+

+-----------------+
|        avg(cost)|
+-----------------+
|91.33333333333333|
+-----------------+



### Milestone 02
1. Find flight routes that have no direct connection

In [23]:
# write your code here

# length 2
routes = g.find("(a)-[]->(b); (b)-[]->(c); !(a)-[]->(c)")
routes.filter("a.id != c.id").show()

# length 3
routes = g.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(d); !(a)-[]->(d)")
routes.filter("a.id != d.id").show()

+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|
+--------------------+--------------------+--------------------+
|[Den Haag, 52.078...|[Hoek van Holland...|[Felixstowe, 51.9...|
|[Utrecht, 52.0928...|[Gouda, 52.01667,...|[Den Haag, 52.078...|
|[Ipswich, 52.0591...|[Felixstowe, 51.9...|[Hoek van Holland...|
|[Gouda, 52.01667,...|[Den Haag, 52.078...|[Hoek van Holland...|
|[Colchester, 51.8...|[London, 51.50986...|[Doncaster, 53.52...|
|[Ipswich, 52.0591...|[Colchester, 51.8...|[London, 51.50986...|
|[Hoek van Holland...|[Felixstowe, 51.9...|[Ipswich, 52.0591...|
|[Hoek van Holland...|[Rotterdam, 51.92...|[Gouda, 52.01667,...|
|[Felixstowe, 51.9...|[Hoek van Holland...|[Rotterdam, 51.92...|
|[Gouda, 52.01667,...|[Den Haag, 52.078...|[Amsterdam, 52.37...|
|[Amsterdam, 52.37...|[Immingham, 53.61...|[Doncaster, 53.52...|
|[Rotterdam, 51.92...|[Hoek van Holland...|[Felixstowe, 51.9...|
|[Felixstowe, 51.9...|[Ho

+--------------------+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|                   d|
+--------------------+--------------------+--------------------+--------------------+
|[Gouda, 52.01667,...|[Rotterdam, 51.92...|[Den Haag, 52.078...|[Hoek van Holland...|
|[Utrecht, 52.0928...|[Gouda, 52.01667,...|[Rotterdam, 51.92...|[Den Haag, 52.078...|
|[Rotterdam, 51.92...|[Den Haag, 52.078...|[Gouda, 52.01667,...|[Utrecht, 52.0928...|
|[Felixstowe, 51.9...|[Hoek van Holland...|[Rotterdam, 51.92...|[Den Haag, 52.078...|
|[Immingham, 53.61...|[Amsterdam, 52.37...|[Den Haag, 52.078...|[Hoek van Holland...|
|[Hoek van Holland...|[Den Haag, 52.078...|[Amsterdam, 52.37...|[Immingham, 53.61...|
|[Hoek van Holland...|[Den Haag, 52.078...|[Rotterdam, 51.92...|[Gouda, 52.01667,...|
|[Den Haag, 52.078...|[Rotterdam, 51.92...|[Gouda, 52.01667,...|[Utrecht, 52.0928...|
|[Felixstowe, 51.9...|[Hoek van Holland...|[Den Haag, 

### Milestone 03
1. Find the most important airport (you can use any measurement to judge the importance level of airports, and please describe why you choose it -the measurement-)

In [21]:
# write your code here

# Most important: the one with the most indegree and outdegree (because it means it has more routes than the other)
airportOutDegrees = g.degrees.sort('degree')
airportOutDegrees.show()

# the answer is Den Haag

+----------------+------+
|              id|degree|
+----------------+------+
|          London|     4|
|      Colchester|     4|
|         Utrecht|     4|
|       Doncaster|     4|
|         Ipswich|     4|
|       Immingham|     4|
|      Felixstowe|     4|
|       Rotterdam|     6|
|       Amsterdam|     6|
|           Gouda|     6|
|Hoek van Holland|     6|
|        Den Haag|     8|
+----------------+------+



### Milestone 04
1. Find the sortest path based on "node" -path with fewest nodes- from Amsterdam to London 

In [15]:
# write your code here
path = g.shortestPaths(landmarks=["London"]).filter("id = 'Amsterdam'")
path.show()

path = g.bfs("id = 'Amsterdam'", "id = 'London'", maxPathLength=5)
path.show(truncate=False)
path.count()

+---------+--------+---------+----------+-------------+
|       id|latitude|longitude|population|    distances|
+---------+--------+---------+----------+-------------+
|Amsterdam|52.37919| 4.899431|    821752|[London -> 3]|
+---------+--------+---------+----------+-------------+



+---------------------------------------+------------------------------------+-------------------------------------+-----------------------------------+---------------------------------------+---------------------------------+---------------------------------------+
|from                                   |e0                                  |v1                                   |e1                                 |v2                                     |e2                               |to                                     |
+---------------------------------------+------------------------------------+-------------------------------------+-----------------------------------+---------------------------------------+---------------------------------+---------------------------------------+
|[Amsterdam, 52.37919, 4.899431, 821752]|[Amsterdam, Immingham, EROAD, 369.0]|[Immingham, 53.61239, -0.22219, 9642]|[Immingham, Doncaster, EROAD, 74.0]|[Doncaster, 53.52285, -1.13116, 302400]|[Doncas

1

### Bonus
1. Find the sortest path based on "cost value" -see 'cost' column in the edge/relationship dataframe - from Amsterdam to London 

In [ ]:
# write your code here

# Use Dijkstra

# Submission
Submit this ```ipynb``` file to the course portal, with format: ```HandsOnWeek13_NIM_NamaLengkap.ipynb```. Make sure when submitting this file, each code cell has the outputs (not blank).